In [1]:
# import pandas as pd
# import numpy as np


# df_transactions = pd.read_csv('data/transactions_data.csv')

# # small version of the dataset

# id_random = np.random.choice(df_transactions['id'].unique(), 1000000, replace=False)
# df_transactions_small = df_transactions[df_transactions['id'].isin(id_random)]

# df_transactions_small.to_csv('data/transactions_data_small.csv', index=False)


In [2]:
import json
import pandas as pd

data = json.load(open('../data/train_fraud_labels.json'))
df_transactions = pd.read_csv('../data/transactions_data_small.csv')

# Yes -> 1, No -> 0

df_transactions["amount"] = df_transactions["amount"].apply(lambda x: x.replace("$", "")).astype(float)
for col in ["use_chip", "merchant_id", "merchant_city", "merchant_state", "mcc"]:
    df_transactions[col] = df_transactions[col].astype('category')


In [3]:
min_frequency = 50
rare_categories = []
for col in ["merchant_id", "merchant_city", "merchant_state", "mcc"]:
    # Calcular las frecuencias de cada categoría
    freq = df_transactions[col].value_counts()
    # Etiquetar como 'others' las categorías que aparecen menos de 'min_frequency' veces
    rc = freq[freq < min_frequency].index
    rare_categories.append(rc)

amount_mean = df_transactions["amount"].mean()
amount_std = df_transactions["amount"].std()



In [4]:
def transform_df(df_transactions,amount_mean,amount_std,rare_categories):

    df_transactions = df_transactions.copy()
    df_transactions.pop("errors")
    df_transactions.pop("card_id")

    df_transactions['is_fraud'] = df_transactions['id'].apply(lambda x: data["target"][str(x)] if str(x) in data["target"] else "unknown")
    df_transactions = df_transactions[df_transactions['is_fraud'] != "unknown"]
    df_transactions['is_fraud'] = df_transactions['is_fraud'].apply(lambda x: 1 if x == "Yes" else 0)

    # if amount is a string, remove the $ and convert to float
    if df_transactions['amount'].dtype == "object":
        df_transactions['amount'] = df_transactions['amount'].apply(lambda x: x.replace("$", "")).astype(float)
        

    df_transactions['amount'] = (df_transactions['amount'] - amount_mean) / amount_std

    # date -> day, month, year and remove date
    df_transactions['date'] = pd.to_datetime(df_transactions['date'])
    df_transactions['day'] = df_transactions['date'].dt.day
    df_transactions['month'] = df_transactions['date'].dt.month
    df_transactions["hour"] = df_transactions["date"].dt.hour
    df_transactions["weekday"] = df_transactions["date"].dt.weekday
    df_transactions.pop("date")

    # id and client_id are not useful
    df_transactions.pop("id")
    df_transactions.pop("client_id")
    df_transactions.pop("zip")

    # Establecer un umbral mínimo de frecuencia
    iter = 0
    for col in ["merchant_id", "merchant_city", "merchant_state", "mcc"]:
        # Calcular las frecuencias de cada categoría
        # Etiquetar como 'others' las categorías que aparecen menos de 'min_frequency' veces
        df_transactions[col] = df_transactions[col].cat.add_categories("others").fillna("others")
        df_transactions[col] = df_transactions[col].replace(rare_categories[iter], "others")

        iter += 1


    return df_transactions

In [5]:
import pandas as pd

In [6]:
df = df_transactions

In [7]:
# Category columns
# use_chip
# merchant_id
# merchant_city
# merchant_state
# mcc



In [8]:
for col in ["use_chip", "merchant_id", "merchant_city", "merchant_state", "mcc"]:
    print(f"{col}: {len(df[col].cat.categories)}")

use_chip: 3
merchant_id: 30050
merchant_city: 9033
merchant_state: 167
mcc: 109


In [9]:
# Establecer un umbral mínimo de frecuencia
min_frequency = 50  # Puedes ajustar este valor según el contexto

for col in ["merchant_id", "merchant_city", "merchant_state", "mcc"]:
    # Calcular las frecuencias de cada categoría
    freq = df[col].value_counts()
    # Etiquetar como 'others' las categorías que aparecen menos de 'min_frequency' veces
    rare_categories = freq[freq < min_frequency].index
    df[col] = df[col].cat.add_categories("others").fillna("others")
    df[col] = df[col].replace(rare_categories, "others")


In [10]:
for col in ["merchant_id", "merchant_city", "merchant_state", "mcc"]:
    print(f"{col}: {len(df[col].cat.categories)}")

merchant_id: 11261
merchant_city: 5214
merchant_state: 122
mcc: 110


In [11]:
df

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499,NaN
13305911,23761869,2019-10-31 23:56:00,1766,2066,12.80,Online Transaction,39261,ONLINE,others,NaN,5815,NaN
13305912,23761870,2019-10-31 23:57:00,199,1031,40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900,NaN
13305913,23761873,2019-10-31 23:58:00,1986,5443,4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411,NaN


In [12]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np



def one_hot_encode(pca,df_original, column):
    # Definir tamaño de los lotes
    batch_size = 10000  # Ajusta según el tamaño que la RAM permita

    # Inicializar lista para almacenar los resultados parciales
    results = []

    # Procesar el DataFrame en lotes
    for i in range(0, len(df_original), batch_size):
        # Crear lote
        df_batch = df_original.iloc[i:i + batch_size].copy()
        
        # Codificación one-hot para el lote
        df_batch = pd.get_dummies(df_batch, columns=[column])
        
        # Aplicar PCA transform en el lote
        result_batch = pca.transform(df_batch)
        
        # Guardar el resultado en la lista
        results.append(result_batch)

    # Concatenar todos los resultados
    result_full = np.vstack(results)
    return result_full

In [13]:

def PCA_one_hot(df,column, n_components,th=0.9):
    # One hot encoding

    df_original = df.copy()

    categories = df[column].cat.categories

    ind_random = np.random.permutation(100000)
    df = pd.DataFrame({column:df_original[column].iloc[ind_random]})

    df[column] = df[column].astype('category')
    df[column] = df[column].cat.set_categories(categories)
    df_one_hot = pd.get_dummies(df, columns=[column])

    # PCA
    pca = PCA(n_components=n_components)
    pca.fit(df_one_hot)
        

    er_cum = np.cumsum(pca.explained_variance_ratio_)
    # find pca.explained_variance_ratio_ > 0.05
    idx = np.where(er_cum > th)
    if  len(idx[0]) > 0:
        idx = idx[0][0]
    else:
        idx = n_components-1
    
    ncateg = df[column].cat.categories.shape[0]

    print(f"n_components = {idx+1} | n_categories = {ncateg}")

    n_components = idx+1


    df_full = pd.DataFrame({column:df_original[column]})
    result_full = one_hot_encode(pca,df_full, column)

    result_full = result_full[:, :n_components]


    df_new = pd.DataFrame(result_full, columns=[column+f"PC{i+1}" for i in range(n_components)])
    df_new.index = df_full.index

    for col in df_new.columns:
        df_original[col] = df_new[col]
    
    df_original.pop(column)

    fig = plt.figure()
    plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
    plt.xlabel("Principal component")
    plt.title("Cumulative explained variance ratio | n_components = "+str(n_components) + " | n_categories = "+str(ncateg) + " | columns = "+column)
    # horizontal line
    plt.axhline(y=th, color='r', linestyle='-')
    # vertical line
    plt.axvline(x=idx, color='r', linestyle='-')
    
    
    return df_original
# 

In [ ]:
# Transformar los datos
for col in ["merchant_id", "merchant_city", "mcc","merchant_state"]:
    df_transactions = PCA_one_hot(df_transactions, col, 50)


n_components = 50 | n_categories = 11261


In [16]:
df_transactions

,amount,use_chip,is_fraud,day,month,hour,weekday,merchant_idPC1,merchant_idPC2,merchant_idPC3,...,merchant_statePC21,merchant_statePC22,merchant_statePC23,merchant_statePC24,merchant_statePC25,merchant_statePC26,merchant_statePC27,merchant_statePC28,merchant_statePC29,merchant_statePC30
2,-0.427342,Swipe Transaction,0,1,1,0,4,-0.174347,-0.008816,-0.027330,...,-0.001514,-0.001077,-0.001587,0.001310,-0.000074,-0.001516,-0.001743,0.000580,-0.000108,-0.001334
3,2.765940,Online Transaction,0,1,1,1,4,-0.180963,-0.010086,-0.031526,...,-0.001497,-0.001066,-0.001570,0.001296,-0.000073,-0.001499,-0.001724,0.000574,-0.000107,-0.001320
4,-0.427950,Swipe Transaction,0,1,1,1,4,0.814349,0.002771,0.007990,...,0.046252,0.037768,0.067130,-0.068634,0.004471,0.143946,0.389756,-0.581514,-0.646470,-0.136825
9,0.281479,Online Transaction,0,1,1,2,4,-0.210758,-0.021771,-0.073668,...,-0.001497,-0.001066,-0.001570,0.001296,-0.000073,-0.001499,-0.001724,0.000574,-0.000107,-0.001320
11,-0.354452,Swipe Transaction,0,1,1,3,4,0.814349,0.002771,0.007990,...,-0.002213,-0.001572,-0.002311,0.001903,-0.000108,-0.002193,-0.002515,0.000836,-0.000156,-0.001913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999992,0.221487,Chip Transaction,0,31,10,21,3,0.814349,0.002771,0.007990,...,-0.008781,-0.006129,-0.008833,0.007144,-0.000400,-0.007967,-0.008933,0.002930,-0.000545,-0.006460
999994,-0.177764,Swipe Transaction,0,31,10,22,3,-0.182097,-0.010330,-0.032341,...,-0.003563,-0.002522,-0.003691,0.003028,-0.000171,-0.003466,-0.003954,0.001310,-0.000244,-0.002973
999995,0.446971,Chip Transaction,0,31,10,22,3,-0.188886,-0.011996,-0.037971,...,0.273743,0.751354,-0.478226,0.168832,-0.007280,-0.094183,-0.079241,0.022852,-0.004086,-0.037223
999997,-0.520553,Chip Transaction,0,31,10,22,3,0.814349,0.002771,0.007990,...,-0.003563,-0.002522,-0.003691,0.003028,-0.000171,-0.003466,-0.003954,0.001310,-0.000244,-0.002973


In [17]:
df_transactions = pd.get_dummies(df_transactions, columns=[ "use_chip"])


In [18]:
df_transactions 

,amount,is_fraud,day,month,hour,weekday,merchant_idPC1,merchant_idPC2,merchant_idPC3,merchant_idPC4,...,merchant_statePC24,merchant_statePC25,merchant_statePC26,merchant_statePC27,merchant_statePC28,merchant_statePC29,merchant_statePC30,use_chip_Chip Transaction,use_chip_Online Transaction,use_chip_Swipe Transaction
2,-0.427342,0,1,1,0,4,-0.174347,-0.008816,-0.027330,-0.060191,...,0.001310,-0.000074,-0.001516,-0.001743,0.000580,-0.000108,-0.001334,False,False,True
3,2.765940,0,1,1,1,4,-0.180963,-0.010086,-0.031526,-0.070892,...,0.001296,-0.000073,-0.001499,-0.001724,0.000574,-0.000107,-0.001320,False,True,False
4,-0.427950,0,1,1,1,4,0.814349,0.002771,0.007990,0.015001,...,-0.068634,0.004471,0.143946,0.389756,-0.581514,-0.646470,-0.136825,False,False,True
9,0.281479,0,1,1,2,4,-0.210758,-0.021771,-0.073668,-0.209931,...,0.001296,-0.000073,-0.001499,-0.001724,0.000574,-0.000107,-0.001320,False,True,False
11,-0.354452,0,1,1,3,4,0.814349,0.002771,0.007990,0.015001,...,0.001903,-0.000108,-0.002193,-0.002515,0.000836,-0.000156,-0.001913,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999992,0.221487,0,31,10,21,3,0.814349,0.002771,0.007990,0.015001,...,0.007144,-0.000400,-0.007967,-0.008933,0.002930,-0.000545,-0.006460,True,False,False
999994,-0.177764,0,31,10,22,3,-0.182097,-0.010330,-0.032341,-0.073022,...,0.003028,-0.000171,-0.003466,-0.003954,0.001310,-0.000244,-0.002973,False,False,True
999995,0.446971,0,31,10,22,3,-0.188886,-0.011996,-0.037971,-0.088230,...,0.168832,-0.007280,-0.094183,-0.079241,0.022852,-0.004086,-0.037223,True,False,False
999997,-0.520553,0,31,10,22,3,0.814349,0.002771,0.007990,0.015001,...,0.003028,-0.000171,-0.003466,-0.003954,0.001310,-0.000244,-0.002973,True,False,False


In [19]:
df_one_hot = df_transactions.copy()

In [20]:
print("Size of the dataset: ", df_one_hot.shape)

Size of the dataset:  (669297, 163)


In [21]:
from sklearn.model_selection import train_test_split

# Variables independientes (X) y dependiente (y)
X = df_one_hot.drop('is_fraud', axis=1)
y = df_one_hot['is_fraud']

# Dividir el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [22]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

# Supongamos que X e y representan tu dataset de características y etiquetas
# Calcular los parámetros de escalado sobre el dataset completo
scaler = StandardScaler()
scaler.fit(X)  # Ajusta el escalador con el dataset completo

# Definir el modelo MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1, warm_start=True)

# Definir las clases del modelo (es importante para partial_fit)
classes = np.unique(y)

# Dividir el dataset en chunks y entrenar con partial_fit
chunk_size = 1000  # Define el tamaño del chunk

def get_chunks(X, y, chunk_size):
    
    id_random = np.random.permutation(chunk_size)
    X = X.iloc[id_random]
    y = y.iloc[id_random]

    return X, y

for i in range(0, len(X), chunk_size):
    
    # Obtener el chunk actual
    X_chunk, y_chunk = get_chunks(X_train, y_train, chunk_size)
    
    # Aplicar la normalización a cada chunk utilizando el escalador ajustado
    X_chunk = scaler.transform(X_chunk)
    
    # Entrenamiento incremental en el chunk actual
    model.partial_fit(X_chunk, y_chunk, classes=classes)
    print(f"Chunk {i} trained on {chunk_size} samples")


Chunk 0 trained on 1000 samples
Chunk 1000 trained on 1000 samples
Chunk 2000 trained on 1000 samples
Chunk 3000 trained on 1000 samples
Chunk 4000 trained on 1000 samples
Chunk 5000 trained on 1000 samples
Chunk 6000 trained on 1000 samples
Chunk 7000 trained on 1000 samples
Chunk 8000 trained on 1000 samples
Chunk 9000 trained on 1000 samples
Chunk 10000 trained on 1000 samples
Chunk 11000 trained on 1000 samples
Chunk 12000 trained on 1000 samples
Chunk 13000 trained on 1000 samples
Chunk 14000 trained on 1000 samples
Chunk 15000 trained on 1000 samples
Chunk 16000 trained on 1000 samples
Chunk 17000 trained on 1000 samples
Chunk 18000 trained on 1000 samples
Chunk 19000 trained on 1000 samples
Chunk 20000 trained on 1000 samples
Chunk 21000 trained on 1000 samples
Chunk 22000 trained on 1000 samples
Chunk 23000 trained on 1000 samples
Chunk 24000 trained on 1000 samples
Chunk 25000 trained on 1000 samples
Chunk 26000 trained on 1000 samples
Chunk 27000 trained on 1000 samples
Chunk

In [23]:

# Predicciones
y_pred = model.predict(X_test)


/home/djoroya/Documents/desafio/hackathon-caixabank-data-ai-report/.conda/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Exactitud del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Exactitud del modelo:", accuracy)


Exactitud del modelo: 0.9984411574281588


In [28]:

# Matriz de confusión
print("Matriz de Confusión:")
mc = confusion_matrix(y_test, y_pred)

print("Verdaderos positivos:", mc[0][0])
print("Falsos positivos:", mc[0][1])
print("Falsos negativos:", mc[1][0])
print("Verdaderos negativos:", mc[1][1])




Matriz de Confusión:
Verdaderos positivos: 200477
Falsos positivos: 0
Falsos negativos: 313
Verdaderos negativos: 0


In [26]:

# Reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))


Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    200477
           1       0.00      0.00      0.00       313

    accuracy                           1.00    200790
   macro avg       0.50      0.50      0.50    200790
weighted avg       1.00      1.00      1.00    200790



/home/djoroya/Documents/desafio/hackathon-caixabank-data-ai-report/.conda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/djoroya/Documents/desafio/hackathon-caixabank-data-ai-report/.conda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/djoroya/Documents/desafio/hackathon-caixabank-data-ai-report/.conda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

In [29]:
data_prediction = json.load(open('../old/predictions_3.json'))

In [32]:
t_id_predict = list(data_prediction["target"].keys())

In [36]:
df_transactions_full = pd.read_csv('../data/transactions_data.csv')

